In [1]:
import sys
from pathlib import Path

# add repo root
root = Path().resolve().parents[0]
sys.path.append(str(root))

In [2]:
from sklearn.model_selection import train_test_split
from src.data.timeseries_dataset import TimeSeriesDataset
from src.config import SENTINEL_DIR, MASK_DIR

all_sentinel_files = list(SENTINEL_DIR.glob("*_RGBNIRRSWIRQ_Mosaic.tif"))
train_ids, val_ids = train_test_split(all_sentinel_files, test_size=0.1, random_state=0)

print(train_ids[0])

C:\Users\emma\Documents\Land-Take-Prediction-Project-NINA-\data\raw\Sentinel\a26-040070508867_45-95719246129747_RGBNIRRSWIRQ_Mosaic.tif


In [3]:
from src.data.transform import ComposeTS, NormalizeBy, RandomCropTS, CenterCropTS

CROP = 64  # use 64 if your tiles are tiny; 128 or 256 only if your tiles are large enough

train_transform = ComposeTS([
    NormalizeBy(10000.0),
    RandomCropTS(CROP),
])

val_transform = ComposeTS([
    NormalizeBy(10000.0),
    CenterCropTS(CROP),          # deterministic for validation
])

train_ds = TimeSeriesDataset(train_ids, sensor="sentinel", slice_mode="first_half", transform=train_transform)
val_ds   = TimeSeriesDataset(val_ids,   sensor="sentinel", slice_mode="first_half", transform=val_transform)


In [4]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=4)
val_loader = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=4)


In [ ]:
import torch

x, mask = next(iter(train_loader))
print(torch.unique(mask))


In [5]:
import torch
ds_raw = TimeSeriesDataset(train_ids, sensor="sentinel", slice_mode="first_half", transform=None)
has_zero = False
has_nonzero = False

for i in range(len(ds_raw)):
    _, m = ds_raw[i]
    u = torch.unique(m)
    if (u == 0).any():
        has_zero = True
    if (u > 0).any():
        has_nonzero = True

print("any tiles with background (0)?", has_zero)
print("any tiles with change (>0)?", has_nonzero)


any tiles with background (0)? True
any tiles with change (>0)? True


In [6]:
x, mask = next(iter(train_loader))
print("x shape:", x.shape)              # (B, T, C, H, W)
print("x range:", x.min().item(), x.max().item())
print("mask shape:", mask.shape)
print("mask unique:", torch.unique(mask))
print("any NaN in x:", torch.isnan(x).any().item())
print("any NaN in mask:", torch.isnan(mask.float()).any().item())


x shape: torch.Size([4, 7, 9, 64, 64])
x range: 0.03660000115633011 0.5397499799728394
mask shape: torch.Size([4, 64, 64])
mask unique: tensor([1])
any NaN in x: False
any NaN in mask: False


In [ ]:
ds_raw = TimeSeriesDataset(train_ids, sensor="sentinel",
                           slice_mode="first_half", transform=None)

x_raw, _ = ds_raw[0]
# x_raw is torch already in your implementation; if not, wrap torch.from_numpy
print("raw x_raw any NaN:", torch.isnan(x_raw).any().item())


In [7]:
import torch
from src.models.external.torchrs_fc_cd import FCEF

device = "cuda" if torch.cuda.is_available() else "cpu"

# probe batch
sample_x, _ = next(iter(train_loader))
_, T, C, H, W = sample_x.shape

model = FCEF(channels=C, t=T, num_classes=2).to(device)
model

FCEF(
  (encoder): Encoder(
    (0): ConvBlock(
      (model): Sequential(
        (0): Conv2d(63, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): Dropout(p=0.2, inplace=False)
      )
      (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): ConvBlock(
      (model): Sequential(
        (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding

In [8]:
from src.models.external.torchrs_fc_cd import FCEF  # or your local file

model1 = FCEF(channels=C, t=T, num_classes=2).to(device)  # C,T from a batch
criterion = torch.nn.CrossEntropyLoss()

x, mask = next(iter(train_loader))
x = x.to(device)
mask = mask.to(device)

model1.eval()
with torch.no_grad():
    logits = model1(x)
    print("logits NaN:", torch.isnan(logits).any().item())
    loss = criterion(logits, mask)
    print("single-batch loss:", loss.item())


logits NaN: False
single-batch loss: 0.6931473612785339


In [9]:
import torch
import torch.nn as nn
from torch.optim import Adam
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1e-3)
scaler = torch.amp.GradScaler("cuda")

for epoch in range(10):
    model.train()
    total_loss = 0.0
    for x, mask in tqdm(train_loader, desc=f"epoch {epoch+1}"):
        x = x.to(device)          # (B, T, C, H, W)
        mask = mask.to(device)    # (B, H, W)

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            logits = model(x)         # (B, 2, H, W)
            loss = criterion(logits, mask)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x, mask in val_loader:
            x = x.to(device)
            mask = mask.to(device)
            with torch.amp.autocast('cuda'):
                logits = model(x)
                loss = criterion(logits, mask)
            val_loss += loss.item()
    val_loss /= len(val_loader)

    print(f"epoch {epoch+1}: train={avg_loss:.4f} val={val_loss:.4f}")


C:\Users\emma\AppData\Local\Temp\ipykernel_19272\240919083.py:8: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  scaler = torch.amp.GradScaler("cuda")
epoch 1:   0%|          | 0/12 [00:00<?, ?it/s]c:\Users\emma\Documents\Land-Take-Prediction-Project-NINA-\prosjekt_venv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
epoch 1: 100%|██████████| 12/12 [00:09<00:00,  1.30it/s]


epoch 1: train=0.7436 val=0.7970


epoch 2: 100%|██████████| 12/12 [00:09<00:00,  1.31it/s]


epoch 2: train=0.6651 val=0.5360


epoch 3: 100%|██████████| 12/12 [00:09<00:00,  1.33it/s]


...

epoch 8: train=0.5069 val=0.4666


epoch 9: 100%|██████████| 12/12 [00:09<00:00,  1.33it/s]


epoch 9: train=0.4988 val=0.4504


epoch 10: 100%|██████████| 12/12 [00:10<00:00,  1.15it/s]


epoch 10: train=0.4901 val=0.4528
